In [ ]:
# import python packages
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
w = 2*np.pi     # natural frequency
d = .25         # damping ratio

# define a linear spring-mass-damper system
A = np.array([[0,1],[-w**2, -2*d*w]])

dt = 0.01    # time step
T = 10       # amount of time to integrate

x0 = [2,0]   # initial condition (x=2, v=0)

# iterate forward Euler 
num_time_slices = int(T/dt)
time_slices = np.linspace(0,T,num_time_slices)
xF = np.zeros((2,num_time_slices))

# remember python is indexed at 0, while matlab is index
xF[:,0] = x0
for k in range(num_time_slices - 1):
    # 2*2 matrix times 2*1 vector,
    # @ symbol is used for a matrix-matrix or matrix-vector multiplication
    xF[:,k+1] = (np.eye(2)+dt*A)@xF[:,k]
    
# iterate backward Euler

xB = np.zeros((2,num_time_slices))
xB[:,0] = x0
for k in range(num_time_slices-1):
    # sholud probably pre-compute inverse for efficiency
    xB[:,k+1] = np.linalg.pinv(np.eye(2)-A*dt)@xB[:,k]

# compute better integral using buit-in scipy function
# 4th-order Runge Kutta

# defining a function for the right-hand side of the ODE
# which scipy wants in case the ODE depends explicitly on t
def ode_func(t,x):
    return A@x

# evaluate ODE function between (0,T) with initial condition
# and return the solution at the times specified by t_eval
ivp_solution = solve_ivp(ode_func,(0,T),x0,t_eval=time_slices)
xRK4_ODE = ivp_solution.y


In [ ]:
def rk4singlestep(fun, dt, t0, y0):
    """
        Long comments describing functions or other complicated
        classes can be left with the triple-quotes notation like this.
        
        This function does a single 4th-order Runge-Kutta step for ODE integration,
        where fun is the ODE, dt is the timestep, t0 is the current time, and y0 is
        the current initial condition. 
    """
    f1 = fun(t0, y0)
    f2 = fun(t0 + dt / 2, y0 + (dt / 2) * f1)
    f3 = fun(t0 + dt / 2, y0 + (dt / 2) * f2)
    f4 = fun(t0 + dt, y0 + dt * f3)
    yRK4 = y0 + (dt / 6) * (f1 + 2 * f2 + 2 * f3 + f4)
    return yRK4

In [ ]:
def Spring_Mass(t, y):
    """
        This function defines the dynamical equations
        that represent the Lorenz system. 
        
        Normally we would need to pass the values of
        sigma, beta, and rho, but we have already defined them
        globally above.
    """
    # y is a three dimensional state-vector
    x,v = y
    dy = [v,-2*d*w*v-w**2*x]
    return np.array(dy)

In [ ]:
y0 = [2,0]

# Compute trajectory 
dt = 0.01
T = 10
num_time_pts = int(T / dt)
t = np.linspace(0, T, num_time_pts)

YRK4 = np.zeros((2, num_time_pts))
YRK4[:, 0] = y0
yin = y0
for i in range(num_time_pts - 1):
    yRK4 = rk4singlestep(Spring_Mass, dt, t[i], yin)
    YRK4[:, i + 1] = yRK4
    yin = yRK4

In [ ]:
plt.plot(time_slices,xF[0,:],'k')
plt.grid(True)
plt.title('x(t)')
plt.plot(time_slices, xB[0,:],'b')
plt.plot(time_slices, xRK4_ODE[0,:],'r')
plt.plot(time_slices, YRK4[0,:],'g')
plt.legend(['Forward Euler','Backward Euler', 'ODE45 (RK4)','RK4'])
plt.show()

In [ ]:
plt.plot(time_slices,xF[1,:],'k')
plt.grid(True)
plt.title('v(t)')
plt.plot(time_slices, xB[1,:],'b')
plt.plot(time_slices, xRK4_ODE[1,:],'r')
plt.plot(time_slices, YRK4[1,:],'g')
plt.legend(['Forward Euler','Backward Euler', 'ODE45 (RK4)','RK4'])
plt.show()